In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

（例）

* 重みを初期化する必要があった
* エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

* Initializerクラスの実装が必要
* Optimizerの実装が必要
* 活性化関数の実装が必要
* ミニバッチサイズの分割が必要
* 損失関数の定義と実装が必要
* 畳み込み層、全結合層などの層のクラスのフォワード・バックの処理の実装が必要

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。

それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="/content/drive/My Drive/read_data/Iris/datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 0, loss : 0.0000, val_loss : 0.9864, acc : 1.000, val_acc : 0.875
Epoch 1, loss : 0.0000, val_loss : 1.6731, acc : 1.000, val_acc : 0.875
Epoch 2, loss : 0.0000, val_loss : 2.1405, acc : 1.000, val_acc : 0.875
Epoch 3, loss : 0.0000, val_loss : 1.3280, acc : 1.000, val_acc : 0.812
Epoch 4, loss : 0.0000, val_loss : 1.9336, acc : 1.000, val_acc : 0.875
Epoch 5, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 6, loss : 0.0000, val_loss : 1.1370, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 5.8109, acc : 1.000, val_acc : 0.750
Epoch 9, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
test_acc : 0.900


<tensorflowでの実装の流れ>

1. ネットワーク構造と各層における演算を定義　⇒　logits
2. 目的関数を定義し、対象のネットワークを組み込む　⇒　loss_op
3. 最適化手法を定義し。最適化を行う関数を指定　⇒　train_op


* データの読み込み・前処理の流れは従来の手法と変わらない
* ミニバッチクラスの実装はtensorflowの学習の前に必要
* ハイパーパラメータの決定はtensorflowでも必要
  * 学習率
  * バッチサイズ
  * エポック数
  * 隠れ層の数とユニット数

* ネットワーク構造の構築はtensorflowでも必要
  * 各層における演算処理の指定、活性化関数の指定、重みの初期値の指定を良きに計らうわけではない

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。

Iris Species

2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。

In [11]:
#前処理工程====================================================================
# データセットの読み込み
dataset_path ="/content/drive/My Drive/read_data/Iris/datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 正規化とワンホット化
ss = StandardScaler().fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

ec = OneHotEncoder(handle_unknown="ignore", sparse=False)
y_train = ec.fit_transform(y_train)
y_test = ec.fit_transform(y_test)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
#==============================================================================

# 各種パラメータの設定==========================================================
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
#==============================================================================

#ネットワーク構築==============================================================
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 初期化クラスの定義
    initializer = tf.keras.initializers.he_normal()
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(initializer([n_hidden1])),
        'b2': tf.Variable(initializer([n_hidden2])),
        'b3': tf.Variable(initializer([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
#==================================================================================

# 計算グラフの実行==============================================================
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.4138, val_loss : 0.7576, acc : 0.833, val_acc : 0.792
Epoch 1, loss : 0.0808, val_loss : 0.4370, acc : 1.000, val_acc : 0.792
Epoch 2, loss : 0.0707, val_loss : 0.3576, acc : 1.000, val_acc : 0.833
Epoch 3, loss : 0.0236, val_loss : 0.2661, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 0.0228, val_loss : 0.2977, acc : 1.000, val_acc : 0.875
Epoch 5, loss : 0.0128, val_loss : 0.2343, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0101, val_loss : 0.2313, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0057, val_loss : 0.2169, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0039, val_loss : 0.2259, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.0030, val_loss : 0.2597, acc : 1.000, val_acc : 0.917
test_acc : 1.000


## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。

この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。

分類問題と回帰問題の違いを考慮してください。

### データ読み込み・前処理

In [72]:
# データセットの読み込み
dataset_path ="/content/drive/My Drive/read_data/HousePrice/train.csv"
df = pd.read_csv(dataset_path)

# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.float)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# 正規化
x_scaler = StandardScaler()
x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
X_val   = x_scaler.transform(X_val)
X_test  = x_scaler.transform(X_test)

#対数変換
y_scaler = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)
y_train = y_scaler.transform(y_train)
y_val   = y_scaler.transform(y_val)
y_test  = y_scaler.transform(y_test)

### ハイパーパラメータの設定

In [73]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_output = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None,1])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

### ネットワーク＆計算グラフの構築

In [76]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    initializer = tf.keras.initializers.he_normal()

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(initializer([n_input, n_hidden1])),
        'w2': tf.Variable(initializer([n_hidden1, n_hidden2])),
        'w3': tf.Variable(initializer([n_hidden2, n_output]))
    }
    biases = {
        'b1': tf.Variable(initializer([n_hidden1])),
        'b2': tf.Variable(initializer([n_hidden2])),
        'b3': tf.Variable(initializer([n_output]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数(MSE)
loss_op = tf.losses.mean_squared_error(labels=Y, predictions=logits)
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 指標値計算
rmse = tf.log(tf.reduce_mean(tf.square(tf.math.expm1(Y)  - tf.math.expm1(logits))))
# variableの初期化
init = tf.global_variables_initializer()

### 計算グラフの実行

In [77]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_score = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, score = sess.run([loss_op, rmse], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_score += score
        total_loss /= n_samples
        total_score /= n_samples
        val_loss, val_score = sess.run([loss_op, rmse], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, score, val_score))
    test_score = sess.run(rmse, feed_dict={X: X_test, Y: y_test})
    print("test_score : {:.3f}".format(test_score))

Epoch 0, loss : 0.0966, val_loss : 0.3363, acc : 21.667, val_acc : 25.482
Epoch 1, loss : 0.0351, val_loss : 0.0940, acc : 20.806, val_acc : 22.141
Epoch 2, loss : 0.0435, val_loss : 0.0760, acc : 21.295, val_acc : 22.081
Epoch 3, loss : 0.0343, val_loss : 0.0666, acc : 21.158, val_acc : 21.788
Epoch 4, loss : 0.0294, val_loss : 0.0638, acc : 20.909, val_acc : 21.747
Epoch 5, loss : 0.0297, val_loss : 0.0631, acc : 20.815, val_acc : 21.638
Epoch 6, loss : 0.0302, val_loss : 0.0664, acc : 20.807, val_acc : 21.689
Epoch 7, loss : 0.0290, val_loss : 0.0728, acc : 20.820, val_acc : 21.741
Epoch 8, loss : 0.0315, val_loss : 0.0783, acc : 20.901, val_acc : 21.786
Epoch 9, loss : 0.0387, val_loss : 0.0907, acc : 21.139, val_acc : 21.941
test_score : 23.799


## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。

3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。

スクラッチで実装したモデルの再現を目指してください。

### データ読み込み・前処理

In [139]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [140]:
#サイズ指定
img_heigt = 28
img_width = 28
feature_size = int(img_heigt * img_width)

#次元削減
X_train = X_train.reshape(-1, feature_size)
X_test  = X_test.reshape(-1, feature_size)

#正規化のために型変換
X_train = X_train.astype(np.float32)
X_test  = X_test.astype(np.float32)

#正規化
X_train /= 255
X_test /= 255

#ワンホット変換
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

print("X shape:", X_train.shape)
print("y shape:", y_train.shape)

X shape: (54000, 784)
y shape: (54000, 10)


各種パラメータの設定

In [147]:
learning_rate = 0.01 #学習率
batch_size = 2700 #バッチサイズ
num_epochs = 10  #エポック数
n_hidden1 = 256  #隠れ層１
n_hidden2 = 128  #隠れ層2
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_output = y_train.shape[1]
# 計算グラフに渡す引数の形を決める
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_output])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [152]:
def network(x):
  initializer = tf.keras.initializers.he_normal()
  #重みとバイアスの宣言
  weights = {
      "w1": tf.Variable(initializer([n_input, n_hidden1])),
      "w2": tf.Variable(initializer([n_hidden1, n_hidden2])),
      "w3": tf.Variable(initializer([n_hidden2, n_output]))
  }

  biases = {
      "b1": tf.Variable(initializer([n_hidden1,])),
      "b2": tf.Variable(initializer([n_hidden2,])),
      "b3": tf.Variable(initializer([n_output,]))
  }

  #順伝播
  layer_1 = tf.matmul(x, weights["w1"]) + biases["b1"]
  layer_1 = tf.nn.relu(layer_1)
  layer_2 = tf.matmul(layer_1, weights["w2"]) + biases["b2"]
  layer_2 = tf.nn.relu(layer_2)
  layer_output = tf.matmul(layer_2, weights["w3"]) + biases["b3"]
  return layer_output

logits = network(X)

#損失関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

#最適化
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(logits, 1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

In [153]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.2997, val_loss : 0.2585, acc : 0.911, val_acc : 0.923
Epoch 1, loss : 0.1890, val_loss : 0.1573, acc : 0.939, val_acc : 0.951
Epoch 2, loss : 0.1344, val_loss : 0.1233, acc : 0.956, val_acc : 0.961
Epoch 3, loss : 0.0990, val_loss : 0.1023, acc : 0.966, val_acc : 0.969
Epoch 4, loss : 0.0753, val_loss : 0.0923, acc : 0.977, val_acc : 0.973
Epoch 5, loss : 0.0561, val_loss : 0.0846, acc : 0.984, val_acc : 0.973
Epoch 6, loss : 0.0461, val_loss : 0.0861, acc : 0.986, val_acc : 0.973
Epoch 7, loss : 0.0346, val_loss : 0.0811, acc : 0.991, val_acc : 0.975
Epoch 8, loss : 0.0380, val_loss : 0.0965, acc : 0.989, val_acc : 0.970
Epoch 9, loss : 0.0268, val_loss : 0.0822, acc : 0.992, val_acc : 0.974
test_acc : 0.975
